In [7]:
import logging
import collections
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open 
import pandas as pd
import numpy as np
from numpy import random
from collections import Counter
import heapq #check n largest
import json
import warnings
from sqlalchemy import create_engine 
import spacy
import nltk
import seaborn as sns

ImportError: No module named 'spacy'

In [ ]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [ ]:
bath_path = r''

computer_security_and_networks = json.load(open(bath_path + r'computer-security-and-networks.json', encoding='utf8'))
algorithms = json.load(open(bath_path + r'algorithms.json', encoding='utf8'))
cloud_computing = json.load(open(bath_path + r'cloud-computing.json', encoding='utf8'))
data_management = json.load(open(bath_path + r'data-management.json', encoding='utf8'))
design_and_product = json.load(open(bath_path + r'design-and-product.json', encoding='utf8'))
electrical_engineering = json.load(open(bath_path + r'electrical-engineering.json', encoding='utf8'))
math_and_logic = json.load(open(bath_path + r'math-and-logic.json', encoding='utf8'))
mobile_and_web_development = json.load(open(bath_path + r'mobile-and-web-development.json', encoding='utf8'))
networking = json.load(open(bath_path + r'networking.json', encoding='utf8'))
probability_and_statistics = json.load(open(bath_path + r'probability-and-statistics.json', encoding='utf8'))
security = json.load(open(bath_path + r'security.json', encoding='utf8'))
machine_learning = json.load(open(bath_path + r'machine-learning.json', encoding='utf8'))
data_analysis = json.load(open(bath_path + r'data-analysis.json', encoding='utf8'))
software_development = json.load(open(bath_path + r'software-development.json', encoding='utf8'))

all_data = computer_security_and_networks + \
algorithms + \
cloud_computing + \
data_management + \
design_and_product + \
electrical_engineering + \
math_and_logic + \
mobile_and_web_development + \
networking + \
probability_and_statistics + \
security + \
machine_learning + \
data_analysis + \
software_development

In [ ]:
len(all_data)

In [ ]:
title=[]; week_data=[]; target_audience=[]; created_by=[]; teach_by=[]
week_data_title=[]; course_info=[]; department=[]; week_data_desc=[]
package_num=[];package_name=[]; level=[]; rating=[]; index=[]
# teach_by, week_data: 多值
for n in range(len(all_data)):
    for i in range(len(all_data[n]['teach_by'])):
            title.append(all_data[n]['title'])
            target_audience.append(all_data[n]['target_audience'])
            created_by.append(all_data[n]['created_by'])
            course_info.append(all_data[n]['course_info'])
            package_num.append(all_data[n]['package_num'])
            package_name.append(all_data[n]['package_name'])
            level.append(all_data[n]['level'])
            rating.append(all_data[n]['rating'])
            teach_by.append(list((all_data[n]['teach_by'][i].values()))[0])  
            department.append(list((all_data[n]['teach_by'][i].values()))[1])

In [ ]:
data = pd.DataFrame({'title': title,
                    'target_audience': target_audience,
                    'created_by' : created_by,  
                    'course_info': course_info,
                    'teach_by': teach_by,
                     'department': department,
                     'level': level,
                     'rating': rating,
                     'package_num': package_num,
                     'package_name': package_name          
                    },
                    columns = ['title', 'course_info', 'target_audience', 'created_by','teach_by',
                               'department','package_num','package_name', 'level','rating'])

In [ ]:
data.head()

In [ ]:
data =  data.drop_duplicates()

In [ ]:
for i, row in data.iterrows():
    if not isEnglish(row.title):
        data.drop(i, inplace=True)

In [ ]:
len(data.title.unique())

In [ ]:
title=[]; created_by=[]; course_info=[]
data_title=[]; data_desc=[]; data_video=[]
# teach_by, week_data: 多值
for n in range(len(all_data)):
    week_data_title=[];  week_data_desc=[];week_data_video=[]
    title.append(all_data[n]['title'])
    created_by.append(all_data[n]['created_by'])
    for i in range(len(all_data[n]['week_data'])):
        week_data_desc.append(all_data[n]['week_data'][i]['description'])
        week_data_title.append(all_data[n]['week_data'][i]['title'])
        week_data_video += all_data[n]['week_data'][i]['video']
    data_title.append(week_data_title)
    data_desc.append(week_data_desc)
    data_video.append(week_data_video)

In [ ]:
data_created_by = pd.DataFrame({'title': title,
                    'created_by':created_by},
                    columns = ['title', 'created_by'])

data_detail = pd.DataFrame({     
                    'week_data_desc': data_desc,
                    'week_data_title': data_title,
                    'week_data_video': data_video
                    },
                    columns = ['week_data_desc', 'week_data_title','week_data_video'])


In [ ]:
data_created_by.drop_duplicates(inplace=True)

In [ ]:
len(data_created_by)

In [ ]:
for i, row in data_created_by.iterrows():
    if not isEnglish(row.title):
        data_created_by.drop(i, inplace=True)

In [ ]:
data_detail = pd.merge(data_created_by, data_detail, left_index=True, right_index=True, how="left")

In [ ]:
for i, row in data_detail.iterrows():
    if len(row.week_data_video) == 0:
        data_detail.drop(i, inplace=True)

In [ ]:
data_detail = data_detail.reset_index(drop=True)

In [ ]:
nlp = spacy.load('en')

In [ ]:
week_video_list = []
for video_data in data_detail.week_data_video:
    video_str = ''
    x = ' '.join(video_data)
    doc = nlp(x.lower())
    for token in doc:
        if not token.is_stop and token.is_alpha:
            video_str += token.lemma_
            video_str += ' '
    week_video_list.append(video_str)

In [ ]:
data_detail['week_data_video_str'] = pd.Series(week_video_list)
data_detail.dropna(inplace=True)
data_detail = data_detail.reset_index(drop=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorize = TfidfVectorizer(max_df=150)
bows = vectorize.fit(data_detail.week_data_video_str)
document_matrix = bows.transform(data_detail.week_data_video_str).toarray()

In [ ]:
document_matrix.shape

In [ ]:
f = bows.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
result = cosine_similarity(document_matrix)

In [ ]:
data_detail.loc[np.argsort(result[20])[::-1], :]

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

svd = TruncatedSVD(100)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X = lsa.fit_transform(document_matrix)
inertia_list = []  
for i in range(2,15):
    kmeans = KMeans(n_clusters=i).fit(X)
    inertia_list.append(kmeans.inertia_)

import matplotlib.pyplot as plt
plt.plot(range(2,15), inertia_list, linestyle='-', marker='o')
plt.show() 

In [ ]:
![image.png](attachment:image.png)

In [ ]:
#IsDuplicated = data_to_mysql.duplicated()

In [ ]:
# duplicate_course=0
# for i in range(len(data_to_mysql)):
#     if IsDuplicated[i] == True:
#         duplicate_course=duplicate_course+1
#         #print(i)
# print(duplicate_course)

In [ ]:
#data = data_to_mysql.drop_duplicates()

In [ ]:
#data_to_mysql.head()

### --------------------------------------------------------------------------------

In [ ]:
### Testing

In [ ]:
# v=[]
# for n in range(len(all_data)):
#     for j in range(len(all_data[n]['week_data'])):     
#         v.append(list((all_data[n]['week_data'][j]).values())[0])
#         #a.append(pd.Series(v).str.cat(sep='')) 

In [ ]:
# v= pd.Series(v).str.cat(sep='')